##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-genai>=1.0.0"
%pip install -q wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [4]:
# Define the model to be used in this notebook
# Note: experimental models have more limited quota for API calls.

model="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

## Function Calling with the Gemini API

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the Search Function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [5]:
from typing import List
import textwrap

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content

        response = client.models.generate_content(
                model=model,
                contents=textwrap.dedent(f"""
                    Extract relevant information
                    about user's query: {query}
                    From this source:

                    {page}

                    Note: Do not summarize. Only Extract and return the relevant information
                """,
            ),
        )

        urls = [url]

        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue
    print()

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results

In [6]:
example = wikipedia_search(["What is a transformer for LLMs?"])

Searching for "What is a transformer for LLMs?"
Related search terms: ['Attention Is All You Need', 'Large language model', 'Transformer (deep learning architecture)']
Fetching page: "Attention Is All You Need"
Information Source: https://en.wikipedia.org/wiki/Attention_Is_All_You_Need
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Transformer (deep learning architecture)"
Information Source: https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)
Transformer (deep learning architecture) did not match with any page id, hence skipping.

Information Sources:
     https://en.wikipedia.org/wiki/Attention_Is_All_You_Need
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)


Here is what the search results look like:

In [7]:
from IPython.display import display, Markdown

for e in example:
  display(Markdown(e))

Here's the extracted information from the provided source, focusing on what a Transformer is in the context of Large Language Models (LLMs):

*   **Underlying Architecture:** The Transformer architecture forms the underlying architecture for most modern Large Language Models (LLMs).

*   **Parallelizability:** A key reason for the Transformer's preference in LLMs is its parallelizability, allowing for faster training times and the ability to train bigger models on GPUs.

*   **Key Mechanisms Introduced:**

    *   **Scaled dot-product attention & self-attention:** The use of the scaled dot-product attention and self-attention mechanism instead of a Recurrent neural network or Long short-term memory.

    *   **Multi-head attention:** Multi-head attention enhances this process by introducing multiple parallel attention heads. Each attention head learns different linear projections of the Q, K, and V matrices. This allows the model to capture different aspects of the relationships between words in the sequence simultaneously, rather than focusing on a single aspect.

    *   **Positional encoding:** The paper relied on the use of sine and cosine wave functions to encode the position of the token into the embedding because the Transformer model is not a seq2seq model and does not rely on the sequence of the text in order to perform encoding and decoding.

*   **Origin:** The Transformer architecture was introduced in the 2017 paper "Attention Is All You Need."

*   **Purpose:** The original focus was on improving seq2seq techniques for machine translation. The removal of recurrence to process all tokens in parallel, while preserving its dot-product attention mechanism to keep its text processing performance. This led to the introduction of a multi-head attention model that was easier to parallelize due to the use of independent heads and the lack of recurrence. Its parallelizability was an important factor to its widespread use in large neural networks.



Based on:
  https://en.wikipedia.org/wiki/Attention_Is_All_You_Need

Here's the extracted information about transformers for LLMs from the provided document:

*   **Transformer Architecture:** Introduced in 2017 by Google researchers in the paper "Attention Is All You Need". It was designed to improve upon previous seq2seq technology and is based on the attention mechanism.
*   **Key Models:** BERT (encoder-only) and GPT (decoder-only) are examples of models based on the transformer architecture.
*   **Dominance:** As of 2024, the largest and most capable LLMs are based on the transformer architecture.
*   **Attention Mechanism:** The attention mechanism is used to find out which tokens are relevant to each other within the scope of the context window by calculating "soft" weights for each token.


Based on:
  https://en.wikipedia.org/wiki/Large_language_model

### Step 2: Pass the System Instructions and Tools to the Model

### Define the prompt

In order to have multiple supporting search queries to the user's original query, you will instruct the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [8]:
instructions = """
    You have access to the Wikipedia API which you will be using
    to answer a user's query. Your job is to generate a list of search queries which
    might answer a user's question. Be creative by using various key-phrases from
    the user's query. To generate variety of queries, ask questions which are
    related to the user's query that might help to find the answer. The more
    queries you generate the better are the odds of you finding the correct answer.
    Here is an example:

    user: Tell me about Cricket World cup 2023 winners.

    function_call: wikipedia_search([
        'What is the name of the team that won the Cricket World Cup 2023?',
        'Who was the captain of the Cricket World Cup 2023 winning team?',
        'Which country hosted the Cricket World Cup 2023?',
        'What was the venue of the Cricket World Cup 2023 final match?',
        'Cricket World cup 2023',
        'Who lifted the Cricket World Cup 2023 trophy?'
    ])

    The search function will return a list of article summaries, use these to
    answer the user's question.
"""

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring as you did above, and then pass the function itself (not a JSON declaration) as a tool. Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In order to yield creative and a more random variety of questions, you could also utilize the model's temperature parameter. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

In [9]:
from google.genai import types

config = types.GenerateContentConfig(
    tools=[wikipedia_search],
    system_instruction=instructions,
) # Pass the function itself

### Step 3: Enable Automatic Function Calling and Call the API (Python SDK Only)

Now start a new chat with default value of `automatic_function_calling`. Using the default `disable=None`, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [11]:
chat = client.chats.create(model=model, config=config, history=[])

query = "Compare dog and cat characteristics."

response = chat.send_message(query)

Searching for "dog characteristics"
Related search terms: ['Dog breed', 'Dog sex', 'Pekingese']
Fetching page: "Dog breed"
Information Source: https://en.wikipedia.org/wiki/Dog_breed
Fetching page: "Dog sex"


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Results when searching for "Dog sex" (originally for "dog characteristics")
        were ambiguous, hence skipping
Fetching page: "Pekingese"
Information Source: https://en.wikipedia.org/wiki/Pekingese

Searching for "cat characteristics"
Related search terms: ['Aegean cat', 'Cat', 'Maine Coon']
Fetching page: "Aegean cat"
Information Source: https://en.wikipedia.org/wiki/Aegean_cat
Fetching page: "Cat"
Information Source: https://en.wikipedia.org/wiki/Cat
Fetching page: "Maine Coon"
Information Source: https://en.wikipedia.org/wiki/Maine_Coon

Searching for "differences between dogs and cats"
Related search terms: ['Cat people and dog people', 'Tabby cat', 'Cat']
Fetching page: "Cat people and dog people"
Information Source: https://en.wikipedia.org/wiki/Cat_people_and_dog_people
Fetching page: "Tabby cat"
Information Source: https://en.wikipedia.org/wiki/Tabby_cat

Searching for "dog behavior"
Related search terms: ['Dog behavior', 'Dog training', 'Dog']
Fetching page: "Dog behavior"

In [12]:
display(Markdown(response.text))

The search results provide information about the characteristics of dogs and cats.
Here's a summary of the key points:

**Dog Characteristics:**

*   Morphology: Body size and shape, tail phenotype, fur type and color, skull shape.
*   Behavioral Traits: Guarding, herding, hunting, retrieving, scent detection.
*   Personality Traits: Hyper-social behavior, boldness, aggression.
*   Temperament: Anxiety and Fear which are linked to gene mutations.
*   Other characteristics: Movement and fitness for purpose.

**Cat Characteristics:**

*   General: Small, domesticated carnivores with strong, flexible bodies, quick reflexes, and sharp teeth.
*   Senses: Well-developed night vision and sense of smell. Can hear sounds too faint or too high for humans.
*   Communication: Includes meowing, purring, trilling, hissing, growling, grunting, and body language. They also secrete and perceive pheromones.
*   Intelligence: Adaptable, learn through observation, and solve problems. Possess strong memories.
*   Coat: A variety of colors that are influenced by the genes MC1R and ASIP.
*   Behavior: Outdoor cats are crepuscular.
*   Grooming: Spend considerable time licking their coats to keep them clean.

The search results also include information about specific breeds such as Pekingese, Aegean cat, Maine Coon and Manx cat.


Check for additional citations:

In [13]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

### Step 4: Understand Chat History

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [14]:
import json
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(part.function_call.name, json.dumps(part.function_call.args, indent=2))
        if part.function_response:
            for res in part.function_response.response['result']:
                display(Markdown(res))
    print("-" * 80)

### user:

Compare dog and cat characteristics.

--------------------------------------------------------------------------------


### model:

wikipedia_search {
  "search_queries": [
    "dog characteristics",
    "cat characteristics",
    "differences between dogs and cats",
    "dog behavior",
    "cat behavior",
    "dog breeds characteristics",
    "cat breeds characteristics"
  ]
}
--------------------------------------------------------------------------------


...

Here's a breakdown of Manx cat characteristics based on the provided text:

**Key Identifying Feature:**

*   **Tail Length:** The most defining characteristic is the variation in tail length, ranging from entirely tailless (rumpy) to having a full-length tail (longy/tailed). Other tail types include riser/rumpy riser, stumpy, and stubby/shorty.

**General Appearance:**

*   **Size:** Medium-sized cats.
*   **Build:** Broad-chested, sloping shoulders, flat sides, muscular, and lean.
*   **Legs:** Hind legs are longer than forelegs, creating a higher rump and arched back, described as rabbit-like.
*   **Head:** Rounded head, medium depth, long neck, large upright ears, large round eyes (often gold variants).

**Coat:**

*   **Type:** Thick, double-layered coat. Two lengths: short and long.
*   **Short-haired:** Dense, soft undercoat and a longer, coarse outer coat. Coat lies close to the skin.
*   **Long-haired (Cymric):** Silky-textured medium-length double coat, with "breeches" (longer fur on the hind legs), belly and neck ruffs, toe tufts, and "ear furnishings" (hairs in ears).
*   **Color & Pattern:** Wide variety of colors and patterns, though original stock had a more limited range (orange, orange and white, cream tabby, tortoiseshell, and rare all-white). Now includes tabby, tortoiseshell, calico, solid colors, marbled, and spotted.

**Temperament & Behavior:**

*   Social, tame, and active.
*   Prized as skilled hunters, especially of rodents.
*   Described as docile, good-tempered, and sociable. Energetic and alert.

**Health Considerations:**

*   **Manx Syndrome/Manxness:** Spinal cord and nerve damage due to shortened spine, leading to spina bifida, bowel/bladder/digestion problems. More common in rumpies.
*   Prone to arthritis in partial tails.
*   Predisposed to rump fold intertrigo (skin irritation) and corneal dystrophy.
*   May develop megacolon (recurring constipation).

**Variants (Sub-breeds):**

*   **Cymric (Manx Longhair):** Long-haired version of the Manx.
*   **Isle of Man Shorthair:** Fully tailed Manx with short hair.
*   **Isle of Man Longhair:** Fully tailed Cymric (long-haired Manx).
*   **Tasman Manx:** Tailless or partially tailed Manx with curly hair.



Based on:
  https://en.wikipedia.org/wiki/Manx_cat

--------------------------------------------------------------------------------


### model:

The search results provide information about the characteristics of dogs and cats.
Here's a summary of the key points:

**Dog Characteristics:**

*   Morphology: Body size and shape, tail phenotype, fur type and color, skull shape.
*   Behavioral Traits: Guarding, herding, hunting, retrieving, scent detection.
*   Personality Traits: Hyper-social behavior, boldness, aggression.
*   Temperament: Anxiety and Fear which are linked to gene mutations.
*   Other characteristics: Movement and fitness for purpose.

**Cat Characteristics:**

*   General: Small, domesticated carnivores with strong, flexible bodies, quick reflexes, and sharp teeth.
*   Senses: Well-developed night vision and sense of smell. Can hear sounds too faint or too high for humans.
*   Communication: Includes meowing, purring, trilling, hissing, growling, grunting, and body language. They also secrete and perceive pheromones.
*   Intelligence: Adaptable, learn through observation, and solve problems. Possess strong memories.
*   Coat: A variety of colors that are influenced by the genes MC1R and ASIP.
*   Behavior: Outdoor cats are crepuscular.
*   Grooming: Spend considerable time licking their coats to keep them clean.

The search results also include information about specific breeds such as Pekingese, Aegean cat, Maine Coon and Manx cat.


--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks questions about cats and dogs.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the searched result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually Execute the Function Call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [15]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True}, # for manual execution
    }

chat    = client.chats.create(model=model, config=config, history=[])
response  = chat.send_message(query)

Initially the model returns a `FunctionCall`:

In [16]:
fc = response.candidates[0].content.parts[0].function_call
print(json.dumps(fc.args, indent=2))

{
  "search_queries": [
    "dog characteristics",
    "cat characteristics"
  ]
}


Call the function with generated arguments to get the results.

In [17]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    display(Markdown("\n ### Function execution result:"))
    for text in summaries:
        display(Markdown(text))

Searching for "dog characteristics"
Related search terms: ['Dog breed', 'Dog sex', 'Pekingese']
Fetching page: "Dog breed"
Information Source: https://en.wikipedia.org/wiki/Dog_breed
Fetching page: "Dog sex"
Results when searching for "Dog sex" (originally for "dog characteristics")
        were ambiguous, hence skipping
Fetching page: "Pekingese"
Information Source: https://en.wikipedia.org/wiki/Pekingese

Searching for "cat characteristics"
Related search terms: ['Aegean cat', 'Cat', 'Maine Coon']
Fetching page: "Aegean cat"
Information Source: https://en.wikipedia.org/wiki/Aegean_cat
Fetching page: "Cat"
Information Source: https://en.wikipedia.org/wiki/Cat
Cat did not match with any page id, hence skipping.
Fetching page: "Maine Coon"
Information Source: https://en.wikipedia.org/wiki/Maine_Coon
Maine Coon did not match with any page id, hence skipping.

Information Sources:
     https://en.wikipedia.org/wiki/Dog_breed
     https://en.wikipedia.org/wiki/Pekingese
     https://en.wik


 ### Function execution result:

**Morphological traits:**
*   Body size and shape
*   Tail phenotype
*   Fur type and colour
*   Skull shape

**Behavioral traits:**

*   Guarding
*   Herding
*   Hunting
*   Retrieving
*   Scent detection

**Personality traits:**

*   Hypersocial behavior
*   Boldness
*   Aggression

**Other Characteristics:**

*   Movement
*   Temperament
*   Form
*   Function
*   Fitness for purpose
*   Phenotype
*   Genetic traits
*   Health issues


Based on:
  https://en.wikipedia.org/wiki/Dog_breed

The Pekingese is a toy dog originating in China, favored by royalty.

**Appearance:**

*   Flat face and large eyes.
*   Compact body, low to the ground.
*   Muscular and durable body.
*   Unusual rolling gait.
*   Slightly longer than tall (ratio of 3 high to 5 long).
*   Weight: 7 to 14 lb (3.2 to 6.4 kg). Smaller ones are called "sleeve" Pekingese.
*   Height: 6–9 inches (15–23 cm) at the withers.

**Coat:**

*   Wide range of color combinations are allowed.
*   Common colors: gold, red, or sable.
*   Less common colors: cream, black, white, tan, black-and-tan, 'blue' or slate grey.
*   Black mask or self-colored face is acceptable.
*   Exposed skin of muzzle, nose, lips, and eye rims is black.
*   Double-coated, requires frequent grooming.

**Health:**

*   Life expectancy of 13.3 years (UK study).
*   Leading cause of death: trauma.
*   Primary health concerns: neurological and cardiovascular defects.
*   Brachycephaly (flattened face) can lead to eye and breathing problems.
*   Other potential concerns: skin allergies (including hotspots), eye ulcers, keratoconjunctivitis sicca (dry eye), and progressive retinal atrophy.

**Care:**

*   Requires daily brushing and grooming every 8–12 weeks.
*   Eyes need to be cleaned daily.
*   Facial creases need to be cleaned to prevent sores.
*   Fur in the rear end needs to be kept clean.
*   Prone to heatstroke, needs to be kept cool.
*   Exercise needs are minimal (around 30 minutes a day).
*   Monitor breathing during exercise.
*   Access to plenty of water is important.


Based on:
  https://en.wikipedia.org/wiki/Pekingese

*   Medium-sized, muscular, semi-longhaired cat.
*   Bicolour or tricolour coat, with white almost always present (25-90% of the body). Other colours and patterns can be present.
*   Medium-sized, round paws.
*   Long, potentially "hooked" tail.
*   Ears with a wide base, rounded tips, covered in hair.
*   Almond-shaped eyes that can be green, blue, or yellow.
*   Affinity for water and fishing.
*   Free from most feline genetic diseases due to natural selection.


Based on:
  https://en.wikipedia.org/wiki/Aegean_cat

Now send the `summaries` to the model.

In [18]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

display(Markdown(response.text))

Okay, I have some information about dog and cat characteristics.

**Dogs:**

*   **Morphological Traits:** Vary greatly by breed, including body size and shape, tail phenotype, fur type and color, and skull shape.
*   **Behavioral Traits:** Include guarding, herding, hunting, retrieving, and scent detection.
*   **Personality Traits:** Can include hypersocial behavior, boldness, and aggression.

**Cats:**

*   **Morphological Traits:** Medium-sized, muscular, semi-longhaired. Bicolour or tricolour coat, with white almost always present (25-90% of the body).
*   **Other possible characteristics**: Medium-sized, round paws. Long, potentially "hooked" tail. Ears with a wide base, rounded tips, covered in hair. Almond-shaped eyes that can be green, blue, or yellow.
*   **Other possible characteristics**: Affinity for water and fishing. Free from most feline genetic diseases due to natural selection.

It's important to note that these are general characteristics, and individual dogs and cats can vary greatly.


## Re-ranking the search results

Helper function to embed the content:

In [19]:
# @title Helper function to embed the content
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to SEMANTIC_SIMILARITY and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and so their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [20]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [21]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [22]:
sim_value = dot_product(search_res, embedded_query)
sim_value

array([[0.78127503],
       [0.63726349],
       [0.71078555]])

using `np.argmax` best candidate is selected.

**User's Input:** Compare dog and cat characteristics.

**Answer:**

In [23]:
display(Markdown(summaries[np.argmax(sim_value)]))

**Morphological traits:**
*   Body size and shape
*   Tail phenotype
*   Fur type and colour
*   Skull shape

**Behavioral traits:**

*   Guarding
*   Herding
*   Hunting
*   Retrieving
*   Scent detection

**Personality traits:**

*   Hypersocial behavior
*   Boldness
*   Aggression

**Other Characteristics:**

*   Movement
*   Temperament
*   Form
*   Function
*   Fitness for purpose
*   Phenotype
*   Genetic traits
*   Health issues


Based on:
  https://en.wikipedia.org/wiki/Dog_breed

### Similarity with User's Preference

Now we can also rank the search results based on your own preference.

In [35]:
ans = input("Are you a dog or cat person? Please respond in 'cat' or 'dog'.\n")

Are you a dog or cat person? Please respond in 'cat' or 'dog'.
dog


Use Gemini API to get embeddings for your answer and compare them with search results

In [36]:
hypothetical_ans = create_embeddings([ans])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [37]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.6438392 ],
       [0.55937082],
       [0.57837405]])

using `np.argmax` best candidate is selected.

**Users's Input:** `dog/ cat`

**Answer:**

In [38]:
display(Markdown(summaries[np.argmax(sim_value)]))

**Morphological traits:**
*   Body size and shape
*   Tail phenotype
*   Fur type and colour
*   Skull shape

**Behavioral traits:**

*   Guarding
*   Herding
*   Hunting
*   Retrieving
*   Scent detection

**Personality traits:**

*   Hypersocial behavior
*   Boldness
*   Aggression

**Other Characteristics:**

*   Movement
*   Temperament
*   Form
*   Function
*   Fitness for purpose
*   Phenotype
*   Genetic traits
*   Health issues


Based on:
  https://en.wikipedia.org/wiki/Dog_breed

You have now created a search re-ranking engine using embeddings based on your own preference!

## Next steps

I hope you found this example helpful! Check out more examples in the [Cookbook](https://github.com/google-gemini/cookbook) to learn more.